### SQL Homework
Use this notebook to answer the questions.
It can be in the same project as the previous homework.  
When you are ready, **upload** to your github repo, and send me the link (just zip a txt file with the repo's address, and upload it as the homework).  
If your repo is private, invite me: balazs.balogh@cubixedu.com.

#### Import the SparkSession, create it then load the taxi data (yellow_tripdata_2024-08.parquet)

In [34]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("SQL")
    .master("local[*]")
    .getOrCreate()
)

taxi_data_2024_08 = (
    spark
    .read
    .format("parquet")
    .load(r"C:\Users\User\Downloads\VibeVoice-main\cubix_data_engineer_pyspark\data\yellow_tripdata_2024-08.parquet")
)

taxi_data_2024_08.createOrReplaceTempView("taxi_2024_08")


#### 1. What is the total fare amount for all trips?  
Please round the answer to two decimal places.

In [20]:
spark.sql("""
    SELECT ROUND(SUM(fare_amount), 2) AS total_fare_amount
    FROM taxi_2024_08
""").show()



+-----------------+
|total_fare_amount|
+-----------------+
|    5.875099806E7|
+-----------------+



#### 2. Show the maximum fare amount, minimum fare amount, and average fare amount for each payment type. Order by payment type.
Round where you need to two decimal places.

In [22]:
spark.sql("""
    SELECT
        payment_type,
        ROUND(MAX(fare_amount), 2) AS max_fare_amount,
        ROUND(MIN(fare_amount), 2) AS min_fare_amount,
        ROUND(AVG(fare_amount), 2) AS avg_fare_amount
    FROM taxi_2024_08
    GROUP BY payment_type
     ORDER BY payment_type
   
""").show()



+------------+---------------+---------------+---------------+
|payment_type|max_fare_amount|min_fare_amount|avg_fare_amount|
+------------+---------------+---------------+---------------+
|           0|         394.76|          -72.2|          19.62|
|           1|          650.0|         -108.7|           20.6|
|           2|         1386.2|        -1174.1|          19.09|
|           3|          999.0|         -999.0|           6.26|
|           4|          900.0|         -900.0|           1.37|
+------------+---------------+---------------+---------------+



#### 3. For trips with a fare amount greater than 20, what is the total tip amount for each day (based on the tpep_pickup_datetime)?
Round the tip to two decimal places, and order the results from highest total tip amount.  
Hint: Check DATE() function, to convert tpep_pickup_datetime to date, to get only the YYYY-MM-DD.

In [29]:
spark.sql("""
    SELECT
        DATE(tpep_pickup_datetime) AS pickup_date,
        ROUND(SUM(tip_amount), 2) AS total_tip_amount
    FROM taxi_2024_08
    WHERE fare_amount > 20
    GROUP BY DATE(tpep_pickup_datetime)
    ORDER BY total_tip_amount DESC
""").show()


+-----------+----------------+
|pickup_date|total_tip_amount|
+-----------+----------------+
| 2024-08-07|       200170.81|
| 2024-08-01|       197694.16|
| 2024-08-29|       192837.64|
| 2024-08-08|       192807.33|
| 2024-08-19|       184872.55|
| 2024-08-22|       182233.24|
| 2024-08-30|       180368.43|
| 2024-08-28|       180220.74|
| 2024-08-05|       176271.55|
| 2024-08-15|        176223.0|
| 2024-08-06|       175451.67|
| 2024-08-11|       174769.89|
| 2024-08-20|       173310.48|
| 2024-08-25|       171774.94|
| 2024-08-04|       169589.92|
| 2024-08-23|        168495.1|
| 2024-08-02|        167908.5|
| 2024-08-14|        166266.4|
| 2024-08-21|       165647.98|
| 2024-08-27|        164666.8|
+-----------+----------------+
only showing top 20 rows



#### 4. For each trip, show the fare amount along with a column that indicates if the trip was "expensive" (greater than 30) or "cheap" (less than or equal to 30).
Hint: Use CASE WHEN for deciding on expensive, or cheap.

In [32]:
spark.sql("""
    SELECT
        fare_amount,
        CASE 
            WHEN fare_amount > 30 THEN 'expensive'
            ELSE 'cheap'
        END AS trip_type
    FROM taxi_2024_08
""").show(10)


+-----------+---------+
|fare_amount|trip_type|
+-----------+---------+
|       28.9|    cheap|
|       40.8|expensive|
|       52.0|expensive|
|       17.0|    cheap|
|        5.1|    cheap|
|        5.8|    cheap|
|       16.3|    cheap|
|       17.7|    cheap|
|       27.5|    cheap|
|       11.4|    cheap|
+-----------+---------+
only showing top 10 rows



#### 5. Find the first trip (based on tpep_pickup_datetime) for each VendorID and display the fare amount.
Hint: You can use CTE with ROW_NUMBER().

In [15]:
spark.sql("""
    WITH first_trips AS (
        SELECT
            VendorID,
            tpep_pickup_datetime,
            fare_amount,
            ROW_NUMBER() OVER (
                PARTITION BY VendorID
                ORDER BY tpep_pickup_datetime ASC
            ) AS rn
        FROM taxi_2024_08
    )
    SELECT
        VendorID,
        tpep_pickup_datetime,
        fare_amount
    FROM first_trips
    WHERE rn = 1
    ORDER BY VendorID
""").show()


+--------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|fare_amount|
+--------+--------------------+-----------+
|       1| 2024-08-01 00:00:04|       70.0|
|       2| 2009-01-01 00:02:52|       10.7|
+--------+--------------------+-----------+



#### 7. Calculate the average trip distance for each VendorID, and assign a label of 'Above Average' or 'Below Average' for each trip based on the distance relative to the VendorID’s average trip distance.
Hint: CTE joined back to the main DataFrame.

In [39]:
spark.sql("""
    WITH avg_distance AS (
        SELECT
            VendorID,
            AVG(trip_distance) AS avg_trip_distance
        FROM taxi_2024_08
        GROUP BY VendorID
    )
    SELECT
        taxi_2024_08.VendorID,
        taxi_2024_08.trip_distance,
        avg_distance.avg_trip_distance,
        CASE
            WHEN taxi_2024_08.trip_distance > avg_distance.avg_trip_distance
                THEN 'Above Average'
            ELSE 'Below Average'
        END AS distance_label
    FROM taxi_2024_08
    JOIN avg_distance
        ON taxi_2024_08.VendorID = avg_distance.VendorID
""").show()


+--------+-------------+-----------------+--------------+
|VendorID|trip_distance|avg_trip_distance|distance_label|
+--------+-------------+-----------------+--------------+
|       1|          7.4|4.034096101093374| Above Average|
|       2|         9.91|5.214354531064327| Above Average|
|       1|         13.4|4.034096101093374| Above Average|
|       1|          3.9|4.034096101093374| Below Average|
|       1|          0.4|4.034096101093374| Below Average|
|       1|          0.4|4.034096101093374| Below Average|
|       2|         3.21|5.214354531064327| Below Average|
|       2|          3.8|5.214354531064327| Below Average|
|       2|         5.54|5.214354531064327| Above Average|
|       2|         1.56|5.214354531064327| Below Average|
|       2|         1.32|5.214354531064327| Below Average|
|       2|         0.45|5.214354531064327| Below Average|
|       2|         2.65|5.214354531064327| Below Average|
|       1|         11.7|4.034096101093374| Above Average|
|       2|    